# Введение в MapReduce модель на Python


In [2]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [3]:
def MAP(_, row:NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)
        
def REDUCE(age:str, rows:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum/count)
    else:
        yield (age, 0)

Модель элемента данных

In [4]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

In [5]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [6]:
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

In [7]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [8]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

In [9]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [11]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [12]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [13]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [14]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [15]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
        
input_collection = [
        User(id=0, age=55, gender='male', social_contacts=20),
        User(id=1, age=25, gender='female', social_contacts=240),
        User(id=2, age=25, gender='female', social_contacts=500),
        User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)
        
def REDUCE(age:str, rows:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum/count)
    else:
        yield (age, 0)
 
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [16]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
    i, j = coordinates
    yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i,j])
            
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.0379767510476)),
 (1, np.float64(2.0379767510476)),
 (2, np.float64(2.0379767510476)),
 (3, np.float64(2.0379767510476)),
 (4, np.float64(2.0379767510476))]

## Inverted index 

In [17]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
    for (docid, document) in enumerate(documents):
        yield ("{}".format(docid), document)
            
def MAP(docId:str, body:str):
    for word in set(body.split(' ')):
        yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
    yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [18]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
    for (docid, document) in enumerate(documents):
        for (lineid, line) in enumerate(document.split('\n')):
            yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
    for word in line.split(" "):  
        yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [19]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()
            
            
def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers
    
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER != None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
    
    print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [20]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps
    
    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid,lineid), line)
            
    split_size =  int(np.ceil(len(documents)/maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
    for word in line.split(" "):  
        yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)
    
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2)])]

## TeraSort

In [21]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
    global maps
    
    def RECORDREADER(split):
        for value in split:
                yield (value, None)
            
    split_size =  int(np.ceil(len(input_values)/maps))
    for i in range(0, len(input_values), split_size):
        yield RECORDREADER(input_values[i:i+split_size])
        
def MAP(value:int, _):
    yield (value, None)
    
def PARTITIONER(key):
    global reducers
    global max_value
    global min_value
    bucket_size = (max_value-min_value)/reducers
    bucket_id = 0
    while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
        bucket_id += 1
    return bucket_id

def REDUCE(value:int, _):
    yield (None,value)
    
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.03723819546990548)),
   (None, np.float64(0.044318385932148874)),
   (None, np.float64(0.08036325344029338)),
   (None, np.float64(0.09211984893489888)),
   (None, np.float64(0.11905744791336048)),
   (None, np.float64(0.14172151248973663)),
   (None, np.float64(0.1673077710265667)),
   (None, np.float64(0.16921896112039547)),
   (None, np.float64(0.19639196696191297)),
   (None, np.float64(0.22954645968125753)),
   (None, np.float64(0.23279492209897945)),
   (None, np.float64(0.24264545455275277)),
   (None, np.float64(0.24870622053052716)),
   (None, np.float64(0.2982273298821676)),
   (None, np.float64(0.36162263499698666)),
   (None, np.float64(0.39961347346689857)),
   (None, np.float64(0.412075652389465)),
   (None, np.float64(0.4754386973596959)),
   (None, np.float64(0.47688639965809454))]),
 (1,
  [(None, np.float64(0.5348847876245267)),
   (None, np.float64(0.5834772189529805)),
   (None, np.float64(0.6860580068813238)),
   (None, np.float64(0.6986

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [22]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

In [23]:
arr1 = np.random.randint(size=(10),low=0,high=10)
arr2 = np.random.randint(size=(10),low=0,high=10)

def RECORDREADER():
    for i in range(2):
        for j in range(arr1.shape[0]):
            yield ((i,j),(arr1,arr2)[i][j])

def MAP(idx, value):
    yield(idx[0], value)
        
 
def REDUCE(idx, iter):
    max = 0
    for i in iter:
        if (max<i): max=i
    yield (idx, max)

print(arr1,'\n',arr2)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[8 5 2 3 5 4 4 0 3 6] 
 [4 6 7 3 5 9 3 6 9 0]
[(0, np.int32(8)), (1, np.int32(9))]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [24]:
arr1 = np.random.randint(size=(10),low=0,high=10)
arr2 = np.random.randint(size=(10),low=0,high=10)

def RECORDREADER():
    for i in range(2):
        for j in range(arr1.shape[0]):
            yield ((i,j),(arr1,arr2)[i][j])

def MAP(idx, value):
    yield(idx[0], value)
        
 
def REDUCE(idx, iter):
    sum = 0
    for i in iter:
        sum+=i
    sum/=len(iter)
    yield (idx, sum)

print(arr1,'\n',arr2)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[9 0 4 3 5 7 6 7 5 7] 
 [9 5 7 3 8 8 7 6 6 9]
[(0, np.float64(5.3)), (1, np.float64(6.8))]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [25]:
def groupbykey(iterable):
    # Сортируем входные данные по ключу
    sorted_iterable = sorted(iterable, key=lambda x: x[0])

    output = dict()

    for (key, val) in sorted_iterable:
        # Добавляем значение к соответствующему ключу
        if key not in output:
            output[key] = list()
        output[key].append(val)

    return output

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

In [26]:
def MAP(_, row:NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum/count)
    else:
        yield (age, 0)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

input_collection = [
    User(id=0, age=55, gender='female', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
print(map_output)
groupbykey(map_output)

[(55, User(id=0, age=55, social_contacts=20, gender='female')), (25, User(id=1, age=25, social_contacts=240, gender='female')), (25, User(id=2, age=25, social_contacts=500, gender='female')), (33, User(id=3, age=33, social_contacts=800, gender='female'))]


{25: [User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')],
 33: [User(id=3, age=33, social_contacts=800, gender='female')],
 55: [User(id=0, age=55, social_contacts=20, gender='female')]}

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [27]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  return reduce_outputs

In [28]:
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d2, d1, d2]


maps = 2
reducers = 3

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        if line.strip():
            yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split():
    if word.strip():
        yield (word, 0)

def REDUCE(key:str, values):
  yield key

print(list(list(INPUTFORMAT())[1]))
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

[('0:1', 'what is it'), ('0:2', 'what is it')]


[(0, ['it']), (1, ['is', 'what']), (2, [])]

# Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [29]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

In [30]:
arr1 = np.random.randint(size=(10),low=0,high=10).tolist()
arr2 = np.random.randint(size=(10),low=0,high=10).tolist()

def pred(t):
    sr = 0
    for i in t:
        sr+=i
    sr/=len(t)
    if (sr > 4.5):return True
    else: return False

def mapp(tuples):
    res = dict()
    for i in range(len(tuples)):
        if pred(tuples[i]):
            res[tuples[i]] = tuples[i]
    return res
        
 
def reduce(elem):
    return (elem)

def record_reader():
    return (tuple(arr1), tuple(arr2))

print(arr1,'\n',arr2)

output = reduce(list(map(mapp, [record_reader()])))
output = list(output)
print(output)

[1, 9, 6, 2, 9, 6, 8, 6, 5, 0] 
 [8, 8, 9, 3, 5, 7, 9, 4, 8, 6]
[{(1, 9, 6, 2, 9, 6, 8, 6, 5, 0): (1, 9, 6, 2, 9, 6, 8, 6, 5, 0), (8, 8, 9, 3, 5, 7, 9, 4, 8, 6): (8, 8, 9, 3, 5, 7, 9, 4, 8, 6)}]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [31]:
arr1 = np.random.randint(size=(10),low=0,high=10).tolist()
arr2 = np.random.randint(size=(10),low=0,high=10).tolist()
S = set([1,2,3])


def exclude_vals(t):
    return tuple([i  for i in t  if i not in S])


def mapp(t):
    tex = exclude_vals(t)
    return (tex,tex)
        
 
def reduce(key, value):
    return (key, key)

def record_reader():
    return (tuple(arr1), tuple(arr2))


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())

print(arr1,'\n',arr2)

output = map(lambda x: reduce(*x),  groupbykey(map(mapp, record_reader())))
output = list(output)
print(output)

[0, 4, 3, 8, 0, 8, 3, 1, 8, 8] 
 [7, 2, 8, 4, 8, 5, 2, 1, 6, 0]
[((0, 4, 8, 0, 8, 8, 8), (0, 4, 8, 0, 8, 8, 8)), ((7, 8, 4, 8, 5, 6, 0), (7, 8, 4, 8, 5, 6, 0))]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [32]:
import random


def mapp(t):
    return (t,t)
        
 
def reduce(key, value):
    return (key, key)

def record_reader():
    return [(random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)) for _ in range(10)]


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())



output = map(lambda x: reduce(*x),  groupbykey(map(mapp, record_reader())))
output = list(output)
print(output)

[((2, 2, 4), (2, 2, 4)), ((3, 3, 2), (3, 3, 2)), ((1, 3, 0), (1, 3, 0)), ((2, 0, 4), (2, 0, 4)), ((0, 0, 4), (0, 0, 4)), ((3, 4, 2), (3, 4, 2)), ((4, 3, 4), (4, 3, 4)), ((4, 0, 0), (4, 0, 0))]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [33]:
def mapp(t):
    return (t,t)
        
 
def reduce(key, value):
    if len(value)==2:
        return (key, key)

def record_reader():
    return [(random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)) for _ in range(20)]


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())



output = map(lambda x: reduce(*x), groupbykey(map(mapp, record_reader())))
output = list(output)
print(output)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [34]:
# True -- R; False -- S

def mapp(t):
    return t
        
 
def reduce(key, value):
    if value[0]:
        return (key, key)

def record_reader():
    return [((random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)), bool(random.randint(0,1))) for _ in range(5)]


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


output = map(lambda x: reduce(*x), groupbykey(map(mapp, record_reader())))
output = list(output)
print(output)

[((3, 0, 1), (3, 0, 1)), None, ((3, 1, 4), (3, 1, 4)), None, None]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [35]:
# True -- R; False -- S


def mapp(key, val):
    if val: key = (key[1], key[0])
    return (key[0],(val,key[1]))
        
 
def reduce(key, values):
    if (len(values) == 2) and (values[0][0]!=values[1][0]):
        res = list()
        a,c = 0, 0

        for val in values:
            if val[0]:
                a = val[1]
            else: 
                c = val[1]
                

        res.append(a)
        res.append(key)
        res.append(c)
        return (res)

def record_reader():
    return [((random.randint(0, 4), random.randint(0, 4)), bool(random.randint(0,1))) for _ in range(5)]


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


# Генерация записей
records = record_reader()

# Применение MAP
map_out = list(map(lambda x: mapp(*x), records))
print("MAP:", map_out)

# Shuffle (группировка по ключу)
shuffle_out = groupbykey(map_out)
print("Group:", shuffle_out)

# Применение REDUCE
reduce_out = [reduce(k, v) for k, v in shuffle_out]
print("Reduce:", reduce_out)

MAP: [(0, (True, 0)), (1, (False, 4)), (4, (True, 3)), (3, (False, 1)), (0, (True, 2))]
Group: [(0, [(True, 0), (True, 2)]), (1, [(False, 4)]), (4, [(True, 3)]), (3, [(False, 1)])]
Reduce: [None, None, None, None]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [36]:
def mapp(t):
    return (t[0], t[1])

def agg(t):
    return sum(t)
 
def reduce(key, value):
    return (key, agg(value))

def record_reader():
    return [(random.randint(0, 4), random.randint(0, 4), random.randint(0, 4)) for _ in range(20)]


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())


output = map(lambda x: reduce(*x), groupbykey(map(mapp, record_reader())))
output = list(output)
print(output)

[(3, 6), (1, 8), (4, 8), (2, 11)]


# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [51]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4)

def MAP(idx, value:int):
    if idx[0] == 'mat':
        i, j = idx[1:]
        yield (i, value*vec[j])

def REDUCE(idx, products:Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (idx, sum)

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield (('mat',i, j), mat[i,j])

    for i in range(vec.shape[0]):
       yield (('vac',i), vec[i])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[(0, np.float64(1.2814756507186698)), (1, np.float64(1.2814756507186698)), (2, np.float64(1.2814756507186698)), (3, np.float64(1.2814756507186698)), (4, np.float64(1.2814756507186698))]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [38]:
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [39]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j,k), big_mat[j,k])
            
def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        yield ((i, k), small_mat[i, j] * w)
    
def REDUCE(key, values):
    (i, k) = key
    yield (key, sum(values))

Проверьте своё решение

In [40]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output)+1
    K = max(k for ((i,k), vw) in reduce_output)+1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [41]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [42]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield (('big',j,k), big_mat[j,k])
    
    for j in range(small_mat.shape[0]):
        for i in range(small_mat.shape[1]):
            yield (('small',j,i), small_mat[j,i])

           
def MAP(k1, v1):
    if k1[0] == 'small':
        j, i = k1[1:]
        w = v1
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[i, k] * w)
    
    
def REDUCE(key, values):
    (i, k) = key
    yield (key, sum(values))

In [43]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

# Verify if the solution matches the reference
print(np.allclose(reference_solution, asmatrix(solution)))  # should return true

True


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [44]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

In [45]:
def flatten(nested_iterable):
    """A function for aligning nested iterable objects."""
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def group_by_key(iterable):
    grouped = {}
    for (key, value) in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

def group_by_key_distributed(map_partitions, partitioner):
    """Distributed grouping by key."""
    global num_reducers
    partitions = [dict() for _ in range(num_reducers)]
    for map_partition in map_partitions:
        for (key, value) in map_partition:
            partition_id = partitions[partitioner(key)]
            partition_id[key] = partition_id.get(key, []) + [value]
    return [(i, sorted(partition.items(), key=lambda x: x[0])) for i, partition in enumerate(partitions)]

def partitioner(obj):
    """Function for determining the batch number."""
    global num_reducers
    return hash(obj) % num_reducers

def MapReduceDistributed(input_format, map_func, reduce_func, partitioner=partitioner, combiner=None):

    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: map_func(*k1v1), record_reader)), input_format())
    if combiner is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: combiner(*k2v2), group_by_key(map_partition))), map_partitions)
    reduce_partitions = group_by_key_distributed(map_partitions, partitioner)
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: reduce_func(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum(len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions]))))
    return reduce_outputs

def as_matrix(reduce_output):
    """Converting the reduction output to a matrix."""
    reduce_output = list(reduce_output)
    rows = max(i for ((i, k), vw) in reduce_output) + 1
    cols = max(k for ((i, k), vw) in reduce_output) + 1
    result_matrix = np.empty(shape=(rows, cols))

    for ((i, k), vw) in reduce_output:
        result_matrix[i, k] = vw

    return result_matrix


In [46]:
def input_format():
    def RECORDREADER_B(prefix):
        big_mat_list = list()
        for j in range(big_mat.shape[0]):
            for i in range(big_mat.shape[1]):
                big_mat_list.append(((prefix,j,i), big_mat[j,i]))
        return big_mat_list

    def RECORDREADER_S(prefix):
        small_mat_list = list()
        for j in range(small_mat.shape[0]):
            for i in range(small_mat.shape[1]):
                small_mat_list.append(((prefix,j,i), small_mat[j,i]))
        return small_mat_list

    yield RECORDREADER_S("small")

    yield RECORDREADER_B("big")


def map_join(key1, value1):

    matrix_id, i, j = key1
    value = value1

    if matrix_id == 'big':
        yield (j, (matrix_id, i, value))
    elif matrix_id == 'small':
        yield (i, (matrix_id, j, value))
    else: pass

def reduce_join(key, values):
    big_matr_values = [v for v in values if v[0] == 'big']
    small_mat_values = [v for v in values if v[0] == 'small']

    for a in big_matr_values:
        for b in small_mat_values:
            yield ((a[1], b[1]), a[2] * b[2])

def get_joined():
    for j in joined:
        print("Join Part:", j)
        yield j[1]

def map_multiply(key1, value1):
    yield (key1, value1)

def reduce_multiply(key, values):
    yield (key, sum(values))

num_maps = 3
num_reducers = 2

partitioned_output = MapReduceDistributed(input_format, map_join, reduce_join)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(get_joined, map_multiply, reduce_multiply)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)


solution_values = []
for p in pre_result:
    solution_values.extend(p[1])

print(solution_values)

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.3830385512615493)), ((0, 1), np.float64(0.22716642291325137)), ((0, 2), np.float64(0.2505250263576772)), ((1, 0), np.float64(0.8105419999645681)), ((1, 1), np.float64(0.4807033812823086)), ((1, 2), np.float64(0.5301321635546604)), ((2, 0), np.float64(0.25240993675415774)), ((2, 1), np.float64(0.14969527806366764)), ((2, 2), np.float64(0.16508783737304916))]), (1, [((0, 0), np.float64(0.16625391015635052)), ((0, 1), np.float64(0.03017572575045533)), ((0, 2), np.float64(0.1300893560343794)), ((1, 0), np.float64(0.3028584109580376)), ((1, 1), np.float64(0.05496996937812684)), ((1, 2), np.float64(0.2369788212143389)), ((2, 0), np.float64(0.7251429731975929)), ((2, 1), np.float64(0.13161624570815855)), ((2, 2), np.float64(0.567405496372482))])]
Join Part: (0, [((0, 0), np.float64(0.3830385512615493)), ((0, 1), np.float64(0.22716642291325137)), ((0, 2), np.float64(0.2505250263576772)), ((1, 0), np.float64(0.81054199996

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [47]:
def input_format():
    def RECORDREADER_B(prefix):
        big_mat_list = list()
        for j in range(big_mat.shape[0]):
            for i in range(big_mat.shape[1]):
                big_mat_list.append(((prefix,j,i), big_mat[j,i]))
        return big_mat_list

    def RECORDREADER_S(prefix):
        small_mat_list = list()
        for j in range(small_mat.shape[0]):
            for i in range(small_mat.shape[1]):
                small_mat_list.append(((prefix,j,i), small_mat[j,i]))
        return small_mat_list

    global num_maps
    split_size = int(np.ceil(big_mat.shape[0]*big_mat.shape[1] / num_maps))

    for i in range(0, big_mat.shape[0]*big_mat.shape[1], split_size):
        yield RECORDREADER_B("big")[i:i + split_size]

    split_size = int(np.ceil(small_mat.shape[0]*small_mat.shape[1] / num_maps))

    for i in range(0, small_mat.shape[0]*small_mat.shape[1], split_size):
        yield RECORDREADER_S("small")[i:i + split_size]


def map_join(key1, value1):

    matrix_id, i, j = key1
    value = value1

    if matrix_id == 'big':
        yield (j, (matrix_id, i, value))
    elif matrix_id == 'small':
        yield (i, (matrix_id, j, value))
    else: pass

def reduce_join(key, values):
    big_matr_values = [v for v in values if v[0] == 'big']
    small_mat_values = [v for v in values if v[0] == 'small']

    for a in big_matr_values:
        for b in small_mat_values:
            yield ((a[1], b[1]), a[2] * b[2])

def get_joined():
    for j in joined:
        print("Join Part:", j)
        yield j[1]

def map_multiply(key1, value1):
    yield (key1, value1)

def reduce_multiply(key, values):
    yield (key, sum(values))

num_maps = 3
num_reducers = 2

partitioned_output = MapReduceDistributed(input_format, map_join, reduce_join)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(get_joined, map_multiply, reduce_multiply)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)


solution_values = []
for p in pre_result:
    solution_values.extend(p[1])

print(solution_values)

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.3830385512615493)), ((0, 1), np.float64(0.22716642291325137)), ((0, 2), np.float64(0.2505250263576772)), ((1, 0), np.float64(0.8105419999645681)), ((1, 1), np.float64(0.4807033812823086)), ((1, 2), np.float64(0.5301321635546604)), ((2, 0), np.float64(0.25240993675415774)), ((2, 1), np.float64(0.14969527806366764)), ((2, 2), np.float64(0.16508783737304916))]), (1, [((0, 0), np.float64(0.16625391015635052)), ((0, 1), np.float64(0.03017572575045533)), ((0, 2), np.float64(0.1300893560343794)), ((1, 0), np.float64(0.3028584109580376)), ((1, 1), np.float64(0.05496996937812684)), ((1, 2), np.float64(0.2369788212143389)), ((2, 0), np.float64(0.7251429731975929)), ((2, 1), np.float64(0.13161624570815855)), ((2, 2), np.float64(0.567405496372482))])]
Join Part: (0, [((0, 0), np.float64(0.3830385512615493)), ((0, 1), np.float64(0.22716642291325137)), ((0, 2), np.float64(0.2505250263576772)), ((1, 0), np.float64(0.81054199996